In [51]:
from sqlalchemy import Column, Integer, String, ForeignKey, create_engine, DateTime, func
from sqlalchemy.orm import declarative_base, Session, relationship
from datetime import datetime, timezone
from typing import List, Tuple, Dict

In [2]:
Base = declarative_base()

class Goods(Base):
    """
    Класс-таблица товаров
    Столбцы:
    id - id товара
    name - название товара(String)
    """
    __tablename__ = 'goods'
    
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)

    purchase = relationship("Purchase", back_populates="goods")
    delivery = relationship("Delivery", back_populates="goods")
    warehouse = relationship("Warehouse", back_populates="goods")

class Sellers(Base):
    """
    Класс-таблица продавцов
    Столбцы:
    id - id продавца
    name_sellers - название продавца(String)
    """
    __tablename__ = 'sellers'
    
    id = Column(Integer, primary_key=True)
    name_sellers = Column(String, nullable=False)

    delivery = relationship("Delivery", back_populates="seller")

class Buyers(Base):
    """
    Класс-таблица покупателей
    Столбцы:
    id - id покупателя
    name_buyer - название покупателя (String)
    """
    __tablename__ = 'buyers'
    
    id = Column(Integer, primary_key=True)
    name_buyer = Column(String, nullable=False)

    purchase = relationship("Purchase", back_populates="buyers")

class Purchase(Base):
    """
    Класс-таблица покупки
    Столбцы:
    id - id покупки
    date - дата покупки(автозаполнение текущей датой) (DateTime)
    buyers_id -id покупателя (Integer)
    goods_id - id товара (Integer)
    quantity - кол-во (Integer)
    price - цена (Integer)
    """

    __tablename__ = 'purchase'
    
    id = Column(Integer, primary_key=True)
    date = Column(DateTime, default=lambda: datetime.now(timezone.utc))  # Автоматически подставляет текущую дату
    buyers_id = Column(Integer, ForeignKey('buyers.id'))
    goods_id = Column(Integer, ForeignKey('goods.id'))
    quantity = Column(Integer, nullable=False)
    price = Column(Integer, nullable=False)

    buyers = relationship("Buyers", back_populates="purchase")
    goods = relationship("Goods", back_populates="purchase")

class Delivery(Base):

    """
    Класс-таблица поставки
    Столбцы:
    id - id поставки
    date - дата поставки(автозаполнение текущей датой) (DateTime)
    sellers_id -id продавца (Integer)
    goods_id - id товара (Integer)
    quantity - кол-во (Integer)
    price - цена (Integer)
    """
    
    __tablename__ = 'delivery'
    
    id = Column(Integer, primary_key=True)
    date = Column(DateTime, default=lambda: datetime.now(timezone.utc))  # Автоматическая текущая дата
    sellers_id = Column(Integer, ForeignKey('sellers.id'))
    goods_id = Column(Integer, ForeignKey('goods.id'))
    quantity = Column(Integer, nullable=False)
    price = Column(Integer, nullable=False)

    seller = relationship("Sellers", back_populates="delivery")
    goods = relationship("Goods", back_populates="delivery")

class Warehouse(Base):

    """
    Класс-таблица склад
    Столбцы:
    id - id складской позиции
    goods_id - id товара (Integer)
    quantity - кол-во (Integer)
    price - цена (Integer)
    """
    
    __tablename__ = 'warehouse'
    
    id = Column(Integer, primary_key=True)
    goods_id = Column(Integer, ForeignKey('goods.id'))
    quantity = Column(Integer, nullable=False)
    price = Column(Integer, nullable=False)

    goods = relationship("Goods", back_populates="warehouse")


# Создаем соединение с базой данных
engine = create_engine("sqlite:///content/mybase.db")



In [3]:
# Base.metadata.create_all(engine)

In [63]:
class ORM:
    
    """
    Класс содержащий методы для работы с базами данных
    Методы:
    add_goods(*names: Tuple[str, ...]) - метод добавления наименований товара
    add_sellers(*name_sellers: Tuple[str, ...]) - метод добавления продавцов
    add_buyers(*name_buyers: Tuple[str, ...]) - метод добавления покупателей
    add_delivery(*deliveries: Dict[str, int]) - метод поставки
    add_purchase(*purchases: Dict[str, int]) - метод покупки
    change_warehouse(type_operation: str, *warehouse_data: Dict[str, int]) - метод для изменения таблицы склад
    sellers_and_buyers_info() - метод вывода инфомрации по покупателям и поставщикам по товарам (использование Union)
    get_warehouse_goods_above(percent: int) - метод вывода скалдких позиций где кол-во выше среднего колва в percent%
    delete_goods(id: int) - метод удаления наименования товара
    delete_sellers(ids: int | str) - метод удаления продавцов
    delete_buyers(ids: int | str) - метод удаления покупателей 
    """
    
    @staticmethod
    def add_goods(*names: Tuple[str, ...]):
        
        """
        Метод  добавления наименований товара
        Параметры:
        *names: Tuple[str, ...] - наименование товара (любое кол-во)
        """
    
        with Session(engine) as session:
            # Создаем объекты товаров на основе переданных названий
            goods_objects = [Goods(name=name) for name in names]
    
            # Добавляем в сессию
            session.add_all(goods_objects)
            session.commit()

    @staticmethod
    def add_sellers(*name_sellers: Tuple[str, ...]):

        """
        Метод  добавления продавца
        Параметры:
        *name_sellers: Tuple[str, ...] - наименование продавца (любое кол-во)
        """
    
        with Session(engine) as session:
            # Создаем список продавцов на основе переданных имен
            sellers_objects = [Sellers(name_sellers=name_seller) for name_seller in name_sellers]
    
            # Добавляем в сессию
            session.add_all(sellers_objects)
            session.commit()

    @staticmethod
    def add_buyers(*name_buyers: Tuple[str, ...]):

        """
        Метод  добавления покупателя
        Параметры:
        *name_buyers: Tuple[str, ...] - наименование покупателя (любое кол-во)
        """
    
        with Session(engine) as session:
            # Создаем список продавцов на основе переданных имен
            buyers_objects = [Buyers(name_buyer=name_buyer) for name_buyer in name_buyers]
    
            # Добавляем в сессию
            session.add_all(buyers_objects)
            session.commit()

    @staticmethod
    def add_delivery(*deliveries: Dict[str, int]):

        """
        Метод  поставки товара
        Параметры:
        *deliveries: Dict[str, int] - словари с данными по доставке (любое кол-во)
                    deliveries["date"],  # Если дата не указана, ставит текущую (автоматически)
                    deliveries["sellers_id"] - id продавца
                    deliveries["goods_id"] - id товара
                    deliveries["quantity"] - кол-во
                    deliveries["price"] - цена
        """
    
        with Session(engine) as session:
            # Создаем объекты на основе переданных данных
            delivery_objects = [
                Delivery(
                    date=data.get("date", datetime.now(timezone.utc)),  # Если дата не указана, ставим текущую
                    sellers_id=data["sellers_id"],
                    goods_id=data["goods_id"],
                    quantity=data["quantity"],
                    price=data["price"]
                ) 
                for data in deliveries
            ]
    
            session.add_all(delivery_objects)  # Добавляем все объектыи в сессию, но не коммитим!
    
             # Обновляем склад
            warehouse_updates = [
                {"goods_id": delivery.goods_id, "quantity": delivery.quantity, "price": delivery.price}
                for delivery in delivery_objects
            ]
            ORM.change_warehouse("delivery", *warehouse_updates)
            session.commit()

    @staticmethod
    def add_purchase(*purchases: Dict[str, int]):

        """
        Метод  продажи толвара
        Параметры:
        *purchases: Dict[str, int] - словари с данными по продаже(ам) (любое кол-во)
                    purchases["date"],  # Если дата не указана, ставит текущую (автоматически)
                    purchases["buyers_id"] - id покупателя
                    purchases["goods_id"] - id товара
                    purchases["quantity"] - кол-во
                    purchases["price"] - цена
        """
    
        with Session(engine) as session:
            # Создаем объекты на основе переданных данных
            purchases_objects = [
                Purchase(
                    date=data.get("date", datetime.now(timezone.utc)),  # Если дата не указана, ставим текущую
                    buyers_id=data["buyers_id"],
                    goods_id=data["goods_id"],
                    quantity=data["quantity"],
                    price=data["price"]
                ) 
                for data in purchases
            ]
    
            session.add_all(purchases_objects)  # Добавляем покупки в сессию, но не коммитим!
    
            # Обновляем склад
            warehouse_updates = [
                {"goods_id": purchase.goods_id, "quantity": purchase.quantity, "price": purchase.price}
                for purchase in purchases_objects
            ]
            ORM.change_warehouse("purchase", *warehouse_updates)
    
            session.commit()  # Один общий коммит после всех операций!


    @staticmethod
    def change_warehouse(type_operation: str, *warehouse_data: Dict[str, int]):

        """
        Метод  работы со складом
        Параметры:
        type_operation - тип задачи продажа или поставка
        *warehouse_data: Dict[str, int] - словарь с данными по продаже(ам)/покупкам (любое кол-во)
                    warehouse_data["buyers_id"]/warehouse_data["sellers_id"] - id покупателя/продавца
                    warehouse_data["goods_id"] - id товара
                    warehouse_data["quantity"] - кол-во
                    warehouse_data["price"] - цена
        """
    
        with Session(engine) as session:
            for data in warehouse_data:
                goods_id = data["goods_id"]
                quantity = data["quantity"]
                price = data["price"]
        
                # Проверяем, есть ли товар на складе
                warehouse_item = (
                    session.query(Warehouse, Goods.name)
                    .join(Goods, Warehouse.goods_id == Goods.id)
                    .filter(Warehouse.goods_id == goods_id)
                    .first()
                )
        
                if warehouse_item:
                    warehouse_record, goods_name = warehouse_item  # Разбиваем кортеж
        
                    if type_operation == "delivery":
                        # Корректное вычисление цены
                        warehouse_record.price = (
                            (warehouse_record.quantity * warehouse_record.price + quantity * price)
                            / (warehouse_record.quantity + quantity)
                        )
                        warehouse_record.quantity += quantity
                    
                    elif type_operation == "purchase":
                        if warehouse_record.quantity >= quantity:
                            warehouse_record.quantity -= quantity
                            if warehouse_record.quantity == 0:
                                ORM.delete_goods(goods_id) 
                        else:
                            raise ValueError(f"Недостаточно товара '{goods_name}' (ID: {goods_id}) на складе!")
                else:
                    # Если товара нет, добавляем новую запись только при поступлении
                    if type_operation == "delivery":
                        new_warehouse_item = Warehouse(goods_id=goods_id, quantity=quantity, price=price)
                        session.add(new_warehouse_item)
                    else:
                        raise ValueError(f"Товар (ID: {goods_id}) отсутствует на складе, покупка невозможна!")

    @staticmethod
    def sellers_and_buyers_info():
        
        """
        Метод для вывода информации по покупателям и продавцам в одноу таблицу (использование Union)
        """
        with Session(engine) as session:
            # Запрос к таблице Sellers с добавлением "Продавец: в имя"
            sellers_query = (
                session.query(
                    func.concat("Продавец: ", Sellers.name_sellers).label("name"),
                    Goods.name.label("goods_name"),
                    func.sum(Delivery.quantity).label("total_quantity"),
                    func.avg(Delivery.price).label("avg_price"),
                )
                .join(Delivery, Delivery.sellers_id == Sellers.id)
                .join(Goods, Delivery.goods_id == Goods.id)
                .group_by(Sellers.name_sellers, Goods.name)
            )
        
            # Запрос к таблице Buyers с добавлением "Покупатель:" в имя
            buyers_query = (
                session.query(
                    func.concat("Покупатель: ", Buyers.name_buyer).label("name"),
                    Goods.name.label("goods_name"),
                    func.sum(Purchase.quantity).label("total_quantity"),
                    func.avg(Purchase.price).label("avg_price"),
                )
                .join(Purchase, Purchase.buyers_id == Buyers.id)
                .join(Goods, Purchase.goods_id == Goods.id)
                .group_by(Buyers.name_buyer, Goods.name)
            )
        
            # Объединяем два запроса
            results = sellers_query.union_all(buyers_query).all()
        
            # Вывод данных
            previous_seller = None
            for row in results:
                if row.name != previous_seller:  # Исправлено имя колонки
                    print(f"{row.name}")
                    previous_seller = row.name
                print(f"  Товар: {row.goods_name}, Общее кол-во: {row.total_quantity}, Средняя цена: {row.avg_price:.2f}")

    @staticmethod
    def get_warehouse_goods_above(percent: int):
        """
        Метод для вывода данных по товарам на складе кол-во которых болье чем percent(процент) от общего 
        кол-ва товара на складе) (использование group_by, having и scalar_subquery (подзапросов))
        Параметры:
        percent от общего кол-ва товаров
        """
        with Session(engine) as session:
            # Подзапрос для вычисления среднего количества товаров на складе
            avg_quantity_subquery = session.query(
                func.avg(Warehouse.quantity)
            ).scalar_subquery()
    
            results = (
                session.query(
                    Goods.name.label("goods_name"),
                    Warehouse.quantity.label("stock_quantity"),
                )
                .join(Goods, Warehouse.goods_id == Goods.id)  # Соединяем склад и товары
                .group_by(Goods.name, Warehouse.quantity)  # Группируем по товарам
                .having(Warehouse.quantity > avg_quantity_subquery * percent/100)  # Фильтр по percent от среднего
                .all()
            )
    
            # Вывод результата
            print(f"Товаров на складе с количеством выше чем {percent}% от общего:")
            for row in results:
                print(f"{row.goods_name}, Кол-во на складе: {row.stock_quantity}")

    @staticmethod
    def delete_goods(id: int):
        """
        Метод для удаления наименования товара (с проверкой по складу, если есть и кол-во не 0, не даем удалить)
        Параметры:
        id товара
        """
        with Session(engine) as session:
            # Проверяем, есть ли товар на складе и его количество 0
            warehouse_query = session.query(Warehouse.goods_id).filter(
                (Warehouse.goods_id == id) & (Warehouse.quantity == 0)
            ).one_or_none()  # Возвращает None, если запись не найдена
    
            if warehouse_query is not None:  # Если товар есть на складе, но его количество = 0
                session.query(Goods).filter(Goods.id == id).delete()
                session.commit()
                print(f"Товар с ID {id} удалён из наименований.")
            else:
                print(f"Товар есть на складе с ненулевым количеством, удалить нельзя.")

    @staticmethod
    def delete_sellers(ids: int | str):
        """
        Метод для удаления поставщика и связанных с ним записей в поставках.
        
        Параметры:
        ids — id (int) или имя (str) поставщика
        """
        with Session(engine) as session:
            # Определяем id поставщика
            if isinstance(ids, int):
                result_sell = session.query(Sellers).filter(Sellers.id == ids).one_or_none()
            elif isinstance(ids, str):
                result_sell = session.query(Sellers).filter(Sellers.name_sellers == ids).one_or_none()
            else:
                raise ValueError("Передан неверный тип данных, ожидается int или str.")
    
            # Проверяем, найден ли поставщик
            if result_sell is None:
                print("Поставщик не найден")
                return
    
            # Удаляем связанные поставки
            session.query(Delivery).filter(Delivery.sellers_id == result_sell.id).delete()
    
            # Удаляем самого поставщика
            session.delete(result_sell)
    
            # Подтверждаем изменения
            session.commit()
    
            print(f"Поставщик '{result_sell.name_sellers}' (ID: {result_sell.id}) удален.")

    @staticmethod
    def delete_buyers(ids: int | str):
        """
        Метод для удаления покупателя и связанных с ним записей в покупках.
        
        Параметры:
        ids — id (int) или имя (str) покупателя
        """
        with Session(engine) as session:
            # Определяем id поставщика
            if isinstance(ids, int):
                result_sell = session.query(Buyers).filter(Buyers.id == ids).one_or_none()
            elif isinstance(ids, str):
                result_sell = session.query(Buyers).filter(Buyers.name_buyer == ids).one_or_none()
            else:
                raise ValueError("Передан неверный тип данных, ожидается int или str.")
    
            # Проверяем, найден ли покупатель
            if result_sell is None:
                print("Покупатель не найден")
                return
    
            # Удаляем связанные покупки
            session.query(Purchase).filter(Purchase.buyers_id == result_sell.id).delete()
    
            # Удаляем самого покупателя
            session.delete(result_sell)
    
            # Подтверждаем изменения
            session.commit()
    
            print(f"Покупатель '{result_sell.name_buyer}' (ID: {result_sell.id}) удален.")

    @staticmethod
    def sellers_and_buyers_soft_info():
        """
        Метод для вывода имен окупателей и продавцов в одну таблицу (использование UNION)
        """
        with Session(engine) as session:
            # Запрос к таблице Sellers с добавлением "Продавец: " в имя
            sellers_query = session.query(func.concat("Продавец: ", Sellers.name_sellers).label("name"))
    
            # Запрос к таблице Buyers с добавлением "Покупатель: " в имя
            buyers_query = session.query(func.concat("Покупатель: ", Buyers.name_buyer).label("name"))
    
            # Объединяем два запроса
            results = sellers_query.union_all(buyers_query).all()
        
   
            for row in results:
                print(f"{row.name}")                 

In [5]:
# ORM.add_goods("Молоко", "Хлеб", "Яблоки", "Кофе", "Мед", "Конфеты", "Винишко", "Бухлишко", "Закусончик")

In [6]:
# ORM.add_sellers("Табак Инвест", "ООО ПрафюмТрэйд", "ООО Белвиллисден", "ООО Белсотра", "АО Приорбанк", "ЗАО Галактика", "ЧП Другой мир", "НПЗ По пятьдесят!", "ЧУП Пенсии и Пляски")

In [7]:
# ORM.add_buyers("Сергей Петрович", "Эрнест Акакиевич", "Зиновий Федорович", "Фекла Аполинарьевна", "Мафусаил Святополкович", "Мэлс Проклович", "Даздраперма Васильевна")

In [24]:
# ORM.add_delivery(
#     {"sellers_id": 1, "goods_id": 1, "quantity": 10, "price": 353245},
#      {"sellers_id": 1, "goods_id": 2, "quantity": 11, "price": 54500},
#      {"sellers_id": 1, "goods_id": 3, "quantity": 12, "price": 456},
#      {"sellers_id": 1, "goods_id": 4, "quantity": 13, "price": 573},
#      {"sellers_id": 1, "goods_id": 5, "quantity": 14, "price": 457},
#      {"sellers_id": 1, "goods_id": 6, "quantity": 15, "price": 457},
#     {"sellers_id": 2, "goods_id": 3, "quantity": 7, "price": 300},
#     {"sellers_id": 3, "goods_id": 3, "quantity": 23, "price": 111},
#     {"sellers_id": 3, "goods_id": 6, "quantity": 11, "price": 589},
#     {"sellers_id": 3, "goods_id": 7, "quantity": 345, "price": 2324},
#     {"sellers_id": 3, "goods_id": 8, "quantity": 23, "price": 356},
#     {"sellers_id": 3, "goods_id": 9, "quantity": 45, "price": 2335},
#     {"sellers_id": 6, "goods_id": 7, "quantity": 234, "price": 2345},
#     {"sellers_id": 7, "goods_id": 3, "quantity": 121, "price": 1343},
# )

In [25]:
# ORM.add_purchase(
#     {"buyers_id": 1, "goods_id": 1, "quantity": 2, "price": 234},
#      {"buyers_id": 1, "goods_id": 2, "quantity": 3, "price": 123},
#      {"buyers_id": 1, "goods_id": 3, "quantity": 5, "price": 345},
#      {"buyers_id": 1, "goods_id": 4, "quantity": 2, "price": 123},
#      {"buyers_id": 1, "goods_id": 5, "quantity": 3, "price": 5621},
#      {"buyers_id": 1, "goods_id": 6, "quantity": 4, "price": 235},
#     {"buyers_id": 2, "goods_id": 3, "quantity": 2, "price": 3545},
#     {"buyers_id": 3, "goods_id": 3, "quantity": 3, "price": 12354},
#     {"buyers_id": 3, "goods_id": 6, "quantity": 4, "price": 785},
#     {"buyers_id": 3, "goods_id": 7, "quantity": 5, "price": 357},
#     {"buyers_id": 3, "goods_id": 8, "quantity": 2, "price": 125},
#     {"buyers_id": 3, "goods_id": 9, "quantity": 1, "price": 637},
#     {"buyers_id": 6, "goods_id": 7, "quantity": 1, "price": 125456},
#     {"buyers_id": 7, "goods_id": 3, "quantity": 2, "price": 124},
# )

In [43]:
ORM.sellers_and_buyers_info()

Продавец: ЗАО Галактика
  Товар: Винишко, Общее кол-во: 234, Средняя цена: 2345.00
Продавец: ООО Белвиллисден
  Товар: Бухлишко, Общее кол-во: 23, Средняя цена: 356.00
  Товар: Винишко, Общее кол-во: 345, Средняя цена: 2324.00
  Товар: Закусончик, Общее кол-во: 45, Средняя цена: 2335.00
  Товар: Конфеты, Общее кол-во: 11, Средняя цена: 589.00
  Товар: Яблоки, Общее кол-во: 23, Средняя цена: 111.00
Продавец: ООО ПрафюмТрэйд
  Товар: Яблоки, Общее кол-во: 7, Средняя цена: 300.00
Продавец: Табак Инвест
  Товар: Конфеты, Общее кол-во: 15, Средняя цена: 457.00
  Товар: Кофе, Общее кол-во: 13, Средняя цена: 573.00
  Товар: Мед, Общее кол-во: 14, Средняя цена: 457.00
  Товар: Молоко, Общее кол-во: 10, Средняя цена: 353245.00
  Товар: Хлеб, Общее кол-во: 11, Средняя цена: 54500.00
  Товар: Яблоки, Общее кол-во: 12, Средняя цена: 456.00
Продавец: ЧП Другой мир
  Товар: Яблоки, Общее кол-во: 121, Средняя цена: 1343.00
Покупатель: Даздраперма Васильевна
  Товар: Яблоки, Общее кол-во: 2, Средняя ц

In [50]:
ORM.get_warehouse_goods_above(10)


Товаров на складе с количеством выше чем 10% от общего:
Бухлишко, Кол-во на складе: 21
Винишко, Кол-во на складе: 573
Закусончик, Кол-во на складе: 44
Конфеты, Кол-во на складе: 18
Кофе, Кол-во на складе: 11
Мед, Кол-во на складе: 11
Яблоки, Кол-во на складе: 151


In [64]:
ORM.sellers_and_buyers_soft_info()

Продавец: Табак Инвест
Продавец: ООО ПрафюмТрэйд
Продавец: ООО Белвиллисден
Продавец: ООО Белсотра
Продавец: АО Приорбанк
Продавец: ЗАО Галактика
Продавец: ЧП Другой мир
Продавец: НПЗ По пятьдесят!
Продавец: ЧУП Пенсии и Пляски
Покупатель: Сергей Петрович
Покупатель: Эрнест Акакиевич
Покупатель: Зиновий Федорович
Покупатель: Фекла Аполинарьевна
Покупатель: Мафусаил Святополкович
Покупатель: Мэлс Проклович
Покупатель: Даздраперма Васильевна


In [65]:
ORM.delete_buyers('Даздраперма Васильевна')

Покупатель 'Даздраперма Васильевна' (ID: 7) удален.


In [66]:
ORM.delete_sellers('АО Приорбанк')

Поставщик 'АО Приорбанк' (ID: 5) удален.


In [67]:
ORM.sellers_and_buyers_soft_info()

Продавец: Табак Инвест
Продавец: ООО ПрафюмТрэйд
Продавец: ООО Белвиллисден
Продавец: ООО Белсотра
Продавец: ЗАО Галактика
Продавец: ЧП Другой мир
Продавец: НПЗ По пятьдесят!
Продавец: ЧУП Пенсии и Пляски
Покупатель: Сергей Петрович
Покупатель: Эрнест Акакиевич
Покупатель: Зиновий Федорович
Покупатель: Фекла Аполинарьевна
Покупатель: Мафусаил Святополкович
Покупатель: Мэлс Проклович
